In [1]:
# 1. 라이브러리 임포트
import os
import shutil
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import torch
from pathlib import Path
import yaml
import pandas as pd
from sklearn.model_selection import train_test_split
from glob import glob

In [2]:
# 2. GPU 또는 CPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cuda


In [3]:
# 3. 모델 및 경로 설정
BASE_DIR = os.path.join(os.getcwd(), "resized")
MODEL_PATH = os.path.join(BASE_DIR, 'best_acne_yolo.pt')  # 2nd_4.ipynb에서 생성된 모델 (YOLO 특징 활용)
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')
YAML_PATH = os.path.join(BASE_DIR, 'data.yaml')
DATASET_DIR = os.path.join(BASE_DIR, "yolo_dataset")
LABEL_DIR = os.path.join(BASE_DIR, 'labels')  # 실제 레이블 파일(.txt)이 있는 폴더

# 출력 및 데이터셋 폴더 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(LABEL_DIR, exist_ok=True)

In [4]:
# 클래스 및 디렉토리 정의
CLASS_NAMES = ['normal', 'mild', 'moderate', 'severe', 'very_severe']
CLASS_ID_MAP = {"normal": 0, "mild": 1, "moderate": 2, "severe": 3, "very_severe": 4}
CLASS_DIRS = {
    "normal": os.path.join(BASE_DIR, "normal"),
    "mild": os.path.join(BASE_DIR, "mild"),
    "moderate": os.path.join(BASE_DIR, "moderate"),
    "severe": os.path.join(BASE_DIR, "severe"),
    "very_severe": os.path.join(BASE_DIR, "very_severe")
}
IMG_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp", ".JPG", ".JPEG", ".PNG")
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = 416  # 학습 및 감지 이미지 크기

# 데이터셋 폴더 생성
train_dir = os.path.join(DATASET_DIR, 'train')
val_dir = os.path.join(DATASET_DIR, 'val')
test_dir = os.path.join(DATASET_DIR, 'test')
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)
# 클래스 및 디렉토리 정의
CLASS_NAMES = ['normal', 'mild', 'moderate', 'severe', 'very_severe']
CLASS_ID_MAP = {"normal": 0, "mild": 1, "moderate": 2, "severe": 3, "very_severe": 4}
CLASS_DIRS = {
    "normal": os.path.join(BASE_DIR, "normal"),
    "mild": os.path.join(BASE_DIR, "mild"),
    "moderate": os.path.join(BASE_DIR, "moderate"),
    "severe": os.path.join(BASE_DIR, "severe"),
    "very_severe": os.path.join(BASE_DIR, "very_severe")
}
IMG_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp", ".JPG", ".JPEG", ".PNG")
NUM_CLASSES = 5
EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = 416  # YOLO 실시간 성능 위해 416 유지 (작으면 속도↑, 크면 정확도↑)
# 데이터셋 폴더 생성
train_dir = os.path.join(DATASET_DIR, 'train')
val_dir = os.path.join(DATASET_DIR, 'val')
test_dir = os.path.join(DATASET_DIR, 'test')
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)

In [5]:
# 4. 이미지 파일 확인 함수
def is_image(p):
    return os.path.isfile(p) and any(p.lower().endswith(ext) for ext in IMG_EXTS)

In [6]:
# 5. 데이터 로드 및 분할 함수 (옵션: 데이터 준비 필요 시 실행)
def load_and_split_data():
    rows = []
    for label_name, class_dir in CLASS_DIRS.items():
        if not os.path.isdir(class_dir):
            print(f"[ERROR] Directory not found: {class_dir}")
            continue
        class_files = [p for p in glob(os.path.join(class_dir, "**", "*"), recursive=True) if is_image(p)]
        if not class_files:
            print(f"[ERROR] No images found in: {class_dir}")
            continue
        print(f"Found {len(class_files)} images in {label_name}")
        for fp in class_files:
            rows.append([fp, label_name, CLASS_ID_MAP[label_name]])
    
    if not rows:
        raise ValueError("No images found in any class directories.")
    
    df = pd.DataFrame(rows, columns=["filepath", "label_name", "label_id"])
    out_csv = os.path.join(BASE_DIR, "labels_total.csv")
    df.to_csv(out_csv, index=False)
    print(f"CSV saved: {out_csv}")
    print("Label distribution:\n", df["label_name"].value_counts())
    
    train_df, test_df = train_test_split(df, test_size=0.15, stratify=df["label_name"], random_state=42)
    train_df, val_df = train_test_split(train_df, test_size=0.1765, stratify=train_df["label_name"], random_state=42)
    print("Train dataset size:", len(train_df))
    print("Val dataset size:", len(val_df))
    print("Test dataset size:", len(test_df))
    
    targets = {"normal": 1000, "mild": 800, "moderate": 800, "severe": 1000, "very_severe": 1000}
    rng = np.random.default_rng(42)
    outs = []
    for lab, n in targets.items():
        sub = train_df[train_df["label_name"] == lab]
        if len(sub) >= n:
            outs.append(sub.sample(n=n, random_state=42))
        else:
            idx = rng.choice(sub.index.to_numpy(), size=n, replace=True)
            outs.append(train_df.loc[idx])
    train_bal = pd.concat(outs, ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_bal.to_csv(os.path.join(BASE_DIR, "split_train.balanced.csv"), index=False)
    val_df.to_csv(os.path.join(BASE_DIR, "split_val.csv"), index=False)
    test_df.to_csv(os.path.join(BASE_DIR, "split_test.csv"), index=False)
    print("Train balanced:\n", train_bal["label_name"].value_counts())
    print("Val:\n", val_df["label_name"].value_counts())
    print("Test:\n", test_df["label_name"].value_counts())
    
    return train_bal, val_df, test_df

In [7]:
# 6. YOLO 데이터셋 폴더 구조 생성 및 실제 레이블 파일 복사
def organize_yolo_folders(csv_df, dest_folder):
    for _, row in csv_df.iterrows():
        label_id = row['label_id']
        src_path = row['filepath']
        img_dest_dir = os.path.join(dest_folder, 'images')
        label_dest_dir = os.path.join(dest_folder, 'labels')
        dest_img_path = os.path.join(img_dest_dir, os.path.basename(src_path))
        dest_label_path = os.path.join(label_dest_dir, os.path.splitext(os.path.basename(src_path))[0] + '.txt')
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_img_path)
            src_label_path = os.path.join(LABEL_DIR, os.path.splitext(os.path.basename(src_path))[0] + '.txt')
            if os.path.exists(src_label_path):
                shutil.copy(src_label_path, dest_label_path)
            else:
                print(f"[WARNING] Label file not found: {src_label_path}")
                if label_id == 0:  # normal 클래스
                    with open(dest_label_path, 'w') as f:
                        f.write("")  # 빈 파일
                else:
                    print(f"[ERROR] Non-normal class missing label: {src_label_path}")

In [8]:
# 7. YOLO 데이터 YAML 파일 생성 (YOLO 학습/감지 설정)
data_yaml = {
    'path': DATASET_DIR,
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': NUM_CLASSES,
    'names': CLASS_NAMES
}
with open(YAML_PATH, 'w') as f:
    yaml.dump(data_yaml, f)
print(f"YAML file created: {YAML_PATH}")

YAML file created: c:\Users\Admin\work space\2nd\resized\data.yaml


In [9]:
# 8. 여드름 감지 함수 (이미지): YOLO 특징 - bounding box로 여드름 부분 표시
def detect_acne_image(sample_path, conf_threshold=0.3, save_output=True):
    results = model(sample_path, conf=conf_threshold)  # YOLO predict: 실시간 bounding box 감지
    for result in results:
        boxes = result.boxes
        print(f"Number of detections: {len(boxes)}")
        for box in boxes:
            cls = int(box.cls)
            conf = float(box.conf)
            xyxy = box.xyxy.cpu().numpy()[0]
            print(f"Detected: {CLASS_NAMES[cls]}, Confidence: {conf:.2f}, Box: {xyxy}")  # 여드름 부분 위치 출력
        if save_output:
            result.save(os.path.join(OUTPUT_DIR, os.path.basename(sample_path)))  # 박스 표시 이미지 저장

In [10]:
# 9. 여드름 감지 함수 (비디오): YOLO 특징 - 프레임별 부분 감지
def detect_acne_video(video_path, conf_threshold=0.3, save_output=True):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"[ERROR] Cannot open video: {video_path}")
        return
    out_path = os.path.join(OUTPUT_DIR, os.path.basename(video_path))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame, conf=conf_threshold)  # 각 프레임에서 여드름 부분 감지
        annotated_frame = results[0].plot()  # bounding box 그리기 (YOLO 특징)
        out.write(annotated_frame)
    cap.release()
    out.release()
    print(f"Video processed and saved to: {out_path}")

In [11]:
# 10. 여드름 감지 함수 (웹캠): YOLO 특징 - 실시간 부분 감지 유지
def detect_acne_webcam(conf_threshold=0.3):
    cap = cv2.VideoCapture(0)  # 웹캠 (0: 기본 카메라)
    if not cap.isOpened():
        print("[ERROR] Cannot open webcam")
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame, conf=conf_threshold)  # 실시간 YOLO 감지: 여드름 부분 bounding box
        annotated_frame = results[0].plot()  # 박스 표시 (클래스, 신뢰도 포함)
        cv2.imshow('Webcam Acne Detection', annotated_frame)  # 창에 표시
        if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키로 종료
            break
    cap.release()
    cv2.destroyAllWindows()

In [12]:
# 11. 모델 로드: 2nd_4.ipynb의 학습된 모델 (YOLO 객체 생성)
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"모델 파일이 없습니다: {MODEL_PATH}. 2nd_4.ipynb를 먼저 실행하세요.")
model = YOLO(MODEL_PATH)  # YOLO 모델 객체: 2nd_4의 pt 파일 로드
model.to(DEVICE)  # GPU/CPU 설정
print("여드름 감지 모델 객체 로드 완료! (YOLO 특징: 실시간 bounding box 감지)")

여드름 감지 모델 객체 로드 완료! (YOLO 특징: 실시간 bounding box 감지)


In [13]:
# 12. 메인 실행
if __name__ == "__main__":
    # 데이터 로드 및 분할 (필요 시 실행, YOLO 데이터셋 준비)
    train_bal, val_df, test_df = load_and_split_data()
    organize_yolo_folders(train_bal, train_dir)
    organize_yolo_folders(val_df, val_dir)
    organize_yolo_folders(test_df, test_dir)
    
    # 테스트 이미지 감지 (예시, YOLO 부분 감지 확인)
    sample_image = os.path.join(BASE_DIR, 'yolo_dataset/test/images/severe_image.jpg')
    if os.path.exists(sample_image):
        print("Processing image...")
        detect_acne_image(sample_image, conf_threshold=0.3, save_output=True)
    else:
        print(f"[WARNING] Sample image not found: {sample_image}")
    
    # 테스트 비디오 감지 (예시)
    sample_video = os.path.join(BASE_DIR, 'test/videos/sample.mp4')
    if os.path.exists(sample_video):
        print("Processing video...")
        detect_acne_video(sample_video, conf_threshold=0.3, save_output=True)
    else:
        print(f"[WARNING] Sample video not found: {sample_video}")
    
    # 웹캠 감지 실행 (YOLO 실시간 특징 활용, 여드름 부분 표시)
    print("Starting webcam detection... Press 'q' to quit.")
    detect_acne_webcam(conf_threshold=0.3)

Found 936 images in normal
Found 359 images in mild
Found 95 images in moderate
Found 128 images in severe
Found 90 images in very_severe
CSV saved: c:\Users\Admin\work space\2nd\resized\labels_total.csv
Label distribution:
 label_name
normal         936
mild           359
severe         128
moderate        95
very_severe     90
Name: count, dtype: int64
Train dataset size: 1124
Val dataset size: 242
Test dataset size: 242
Train balanced:
 label_name
very_severe    1000
normal         1000
severe         1000
moderate        800
mild            800
Name: count, dtype: int64
Val:
 label_name
normal         141
mild            54
severe          19
very_severe     14
moderate        14
Name: count, dtype: int64
Test:
 label_name
normal         141
mild            54
severe          19
very_severe     14
moderate        14
Name: count, dtype: int64
[WARNING] Label file not found: c:\Users\Admin\work space\2nd\resized\labels\levle3_84.txt
[ERROR] Non-normal class missing label: c:\Users\Ad